# NeuralNet to classify Ising states

In [52]:
import os
from glob import glob

import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Preparing the NN inputs
TODO: decide which labels to use; return it in generate_batch()

In [ ]:
def get_temperature(filename):
    temperature = filename.split('_')[0].replace('T', '')
    return float(temperature)

data_dir = 'data/square_ising_configs_1600spins/'
filenames = os.listdir(data_dir)
temperatures = list({get_temperature(fn) for fn in filenames})
temperatures.sort()

In [51]:
def load_spins(file):
    spins = np.load(file)
    side = int(np.sqrt(len(spins)))
    spins = spins.reshape((side, side))
    return spins

def generate_batch(data_dir, temperature, batch_size):
    # TODO: return labels as well!
    files = glob(os.path.join(data_dir, 'T{:.3f}*'.format(temperature)))
    i = 0
    while i+batch_size < len(files):
        spins = np.array([load_spins(file) for file in files[i: i+batch_size]])
        i += batch_size
        yield spins

In [49]:
data_dir = 'data/square_ising_configs_1600spins/'
temperature = temperatures[0]
batch_size = 10
trainloader = generate_batch(data_dir, temperature, batch_size)

### Building the NN

In [2]:
class Net(nn.Module):
    
    def __init__(self, a=40, kernel_size=4, n_hidden=2, n_out=2):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1, 1, kernel_size)
        # h_out = int((h_in + 2*h_padding - h_dilatation*(h_kernel-1) -1) / h_stride + 1)
        a_out = a - kernel_size + 1 # n_spins = a**2
        self.fc1 = nn.Linear(a_out**2, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_out)
        
    def forward(self, x):
        x = self.conv(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return x
        
net = Net()

In [3]:
net

Net (
  (conv): Conv2d(1, 1, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear (1369 -> 2)
  (fc2): Linear (2 -> 2)
)

In [4]:
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

### Training

In [50]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[{:d}, {:5d}] loss: {:.3f}'.format(epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

ValueError: too many values to unpack (expected 2)